In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta, timezone

from my_code._library import CrawlingWeb
from my_code._library import MyLibrary
lib = MyLibrary(r"/Users/jacklucian/Python/my_code")
ggf = lib.google
bqr = lib.bigquery
udf = lib.function
query = f"""
select
  date_trunc(event_date, month) as date_key,
  creator_id,
  feature,
  count(distinct event_code) as event_count
from `tevi-data.tevi_data_team.ops_fact__total_event`
where true
  and feature in ('live', 'post')
  and follower_count_group is not null
group by all
"""
df = bqr.run_biqquery_to_df(query=query)
df['date_key'] = pd.to_datetime(df['date_key'], errors='coerce').dt.date

df_with_flag = (
    df
    .groupby(['date_key', 'feature'])[df.columns.tolist()]
    .apply(
        lambda group: (
            group
            .assign(
                q75=np.percentile(group['event_count'], 75),
                q25=np.percentile(group['event_count'], 25)
            )
            .assign(
                iqr=lambda x: x['q75'] - x['q25'],
                upper_bound=lambda x: x['q75'] + 1.5 * x['iqr'],
                lower_bound=lambda x: np.maximum(0, x['q25'] - 1.5 * x['iqr']),
                flag=lambda x:
                    np.where(x['event_count'] >= x['upper_bound'], 11,
                    np.where(x['event_count'] <= x['lower_bound'], -1, 0)
                    ),
                quantile=lambda x:
                    np.where(x['flag'] == 11, x['event_count'].rank(method='min', pct=True).apply(lambda pct: int(pct * 10)),
                    np.where(x['flag'] == 0 , x['event_count'].rank(method='min', pct=True).apply(lambda pct: int(pct * 10)), x['flag'])
                             
            )
            )
            .drop(columns=['q75', 'q25', 'iqr', 'upper_bound', 'lower_bound', 'date_key', 'feature'])
        )
        , include_groups=False
    )
    .reset_index(level=[0, 1])
)
bqr.update_table_from_dataframe(
    df=df_with_flag,
    write_disposition='WRITE_TRUNCATE',
    partition_column_name='date_key',
    partition_column_type='DAY',
    table_id='tevi-data.tevi_data_team.ops_sub__creator_with_quantile_buckets'
)
